##Functions

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applySPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = SparsePCA(n_components=numComponents)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def applyMRMR(X,y,K):
    from mrmr import mrmr_classif

    X_q = X.reshape(np.prod(X.shape[:2]),np.prod(X.shape[2:]))
    y_q = y.reshape(np.prod(y.shape[:2]))

    X = pd.DataFrame(X_q)
    y = pd.Series(y_q)

    newX = np.reshape(X, (-1, X.shape[1]))
    selected_features = mrmr_classif(newX, y, K)

    X = X[:,:,selected_features]

    return X

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=25, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
loc_1 = "/content/drive/MyDrive/Colab Notebooks/Paper 05/"
loc_2 = "06_SPCA_3D_2D_CNN_"

dataset = 'SA'
test_ratio = 0.9
windowSize = 25

ratio_per = 100-int(test_ratio*100)
ratio_per = str(ratio_per)+'%'
ratio_per

'10%'

In [ ]:
X, y = loadData(dataset)

X.shape, y.shape

((512, 217, 204), (512, 217))

In [ ]:
# Visualize original data
K = X.shape[2]
X_vis = X.reshape(np.prod(X.shape[:2]),np.prod(X.shape[2:]))
df = pd.DataFrame(X_vis,columns=(list(range(1,K+1))))
df

##Apply SPCA

In [ ]:
# K = 10 if dataset == 'IP' else 5
K = X.shape[2]
X,pca = applySPCA(X,numComponents=K)

X.shape

In [ ]:
K = 10 if dataset == 'IP' else 5
X = applyMRMR(X,y,K)
X.shape


In [ ]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

Xtrain, Xvalid, ytrain, yvalid = splitTrainTestSet(Xtrain, ytrain, 0.3333)

Xtrain.shape, Xvalid.shape, ytrain.shape, yvalid.shape

# Model and Training

In [ ]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

In [ ]:
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)
ytrain.shape, ytest.shape

Xvalid = Xvalid.reshape(-1, windowSize, windowSize, K, 1)
Xvalid.shape

yvalid = np_utils.to_categorical(yvalid)
yvalid.shape

In [ ]:
S = windowSize
L = K
# output_units = 9 if (dataset == 'PU' or dataset == 'PC') else 16
output_units = ytrain.shape[1]

In [ ]:


input_shape = (15, 15, L, 1)
output_units = 14

input_layer = Input(shape=input_shape)

# First branch
conv_layer1_1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
norm_1_1 = BatchNormalization()(conv_layer1_1)

conv_layer1_2 = Conv3D(filters=16, kernel_size=(3, 3, 1), activation='relu', kernel_regularizer=l2(0.0001))(norm_1_1)
norm_1_2 = BatchNormalization()(conv_layer1_2)

# Second Branch
conv_layer2_1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
norm_2_1 = BatchNormalization()(conv_layer2_1)

conv_layer2_2 = Conv3D(filters=16, kernel_size=(3, 3, 1), activation='relu', kernel_regularizer=l2(0.0001))(norm_2_1)
norm_2_2 = BatchNormalization()(conv_layer2_2)


concat = concatenate([norm_1_2, norm_2_2])

conv3d_shape = concat.shape
conv_layer___ = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(concat)

conv_layer5 = DepthwiseConv2D(kernel_size=(3, 3), activation='relu')(conv_layer___)
norm_5 = BatchNormalization()(conv_layer5)

flatten_layer = Flatten()(norm_5)

dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)

dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.5)(dense_layer2)

output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:
# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

# model_arch = f'{loc_1}figure/{ratio_per}/{dataset}/{loc_2}architech{dataset}.png'
# plot_model(model, to_file=model_arch)

model.summary()

In [ ]:
# compiling the model
adam = Adam(learning_rate=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
# checkpoint
filepath = "/content/drive/MyDrive/Colab Notebooks/Paper 05/saved_model/default_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=120, callbacks=callbacks_list)

In [ ]:
score = model.evaluate(Xtest,ytest,verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

# Save the model

In [ ]:
plt.figure(figsize=(6,6))
plt.grid()
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'], loc='upper right')
plt.savefig(f'{loc_1}figure/{ratio_per}/{dataset}/{loc_2}loss_curve{dataset}.png')
# plt.show()

plt.figure(figsize=(6,6))
# plt.ylim(0,1.1)
plt.grid()
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'], loc='lower right')
plt.savefig(f'{loc_1}figure/{ratio_per}/{dataset}/{loc_2}acc_curve{dataset}.png')
# plt.show()

In [ ]:
import os.path

if os.path.isfile("/content/drive/MyDrive/Colab Notebooks/Paper 05/saved_model/default_model.hdf5") is False:
    model.save("/content/drive/MyDrive/Colab Notebooks/Paper 05/saved_model/default_model.hdf5")

# Validation

In [ ]:
# load best weights
model.load_weights("/content/drive/MyDrive/Colab Notebooks/Paper 05/saved_model/default_model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
Xtest = Xtest.reshape(-1, windowSize, windowSize, K, 1)
Xtest.shape

In [ ]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [ ]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
def reports (X_test,y_test,name,cm_img_loc):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed',
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    elif name == 'BW':
        target_names = ['water','hippo grass', 'floodplain grasses 1', 'floodplain grasses 2', 'reeds1',
                        'riparian', 'firescar2', 'island interior', 'acacia woodlands', 'acacia shrublands',
                        'acacia grasslands', 'short mopane', 'mixed mopane', 'exposed soils']

    classification = classification_report(np.argmax(y_test, axis=1),y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1),y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1),y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1),y_pred)
    score = model.evaluate(X_test, y_test)
    Test_accuracy = score*100

    # CLasswise accuracy measure --------------->
    row_col = [f"C{i}" for i in range(1,confusion.shape[0]+1)]
    cm_pd = pd.DataFrame(confusion, columns = row_col, index = row_col)

    cm_mid = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
    CA = cm_mid.diagonal()
    CA_ = pd.DataFrame(CA, columns = ['C.A'],index = row_col)

    cm_pd = pd.concat([cm_pd,CA_], axis = 1)


    # Confusion matrix visualize ------------------->
    df_cm = pd.DataFrame(confusion, columns=np.unique(target_names), index = np.unique(target_names))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (16,10))
    sn.set(font_scale=1.4)#for label size
    sn.heatmap(df_cm, cmap="PuBu", annot=True,annot_kws={"size": 16}, fmt='d')

    plt.savefig(cm_img_loc, bbox_inches='tight', dpi=300)
    plt.close()

    return classification, cm_pd, Test_accuracy, oa*100, aa*100, kappa*100

In [ ]:
file_name = f"{loc_1}reports/{ratio_per}/{dataset}/{loc_2}classification_report{dataset}.txt"
file_name2 = f"{loc_1}reports/{ratio_per}/{dataset}/{loc_2}classwise_accuracy{dataset}.csv"
file_name3 = f"{loc_1}figure/{ratio_per}/{dataset}/{loc_2}CM_img{dataset}.png"

classification, confusion, Test_accuracy, oa, aa, kappa = reports(Xtest,ytest,dataset,file_name3)
# print(classification)
classification = str(classification)
# confusion = str(confusion)

confusion.to_csv(file_name2)

with open(file_name, 'w') as x_file:
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    # x_file.write('\n\nConfusion matrix:\n')
    # x_file.write('{}'.format(confusion))




1523/1523 [==============================] - 7s 4ms/step - loss: 0.0027 - accuracy: 0.9992


In [ ]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]

    return patch

In [ ]:
# load the original image
X, y = loadData(dataset)

In [ ]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize

In [ ]:
K = X.shape[2]
X,pca = applySPCA(X, numComponents=K)
K = 10 if dataset == 'IP' else 5
X = MIFS(X,y,K)
X.shape

In [ ]:
X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')
            prediction = (model.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1

In [ ]:
ground_truth = spectral.imshow(classes = y,figsize =(5,5))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(5,5))

In [ ]:
Cmap_loc = f"{loc_1}figure/{ratio_per}/{dataset}/{loc_2}classificaiton_map{dataset}.png"
spectral.save_rgb(Cmap_loc, outputs.astype(int), colors=spectral.spy_colors)

spectral.save_rgb(str(dataset)+"_ground_truth.jpg", y, colors=spectral.spy_colors)